In [17]:
import LingAnalysis as LA
import os, os.path, xmltodict, re

In [4]:
# Load data

path_labels = "/Users/morales/Desktop/clpsych17-data/data/training/labels.tsv"

with open(path_labels) as f:
    labels = [line.strip().split('\t') for line in f.readlines()]

id_labels = {}
for line in labels:
    post_id = line[0]
    post_label = line[1]
    id_labels[post_id] = post_label


green = id_labels.values().count('green')
amber = id_labels.values().count('amber')
crisis = id_labels.values().count('crisis')
red = id_labels.values().count('red')


print "GREEN POSTS:", green
print "AMBER POSTS:", amber
print "RED POSTS:", red
print "CRISIS POSTS:", crisis

GREEN POSTS: 715
AMBER POSTS: 296
RED POSTS: 137
CRISIS POSTS: 40


In [ ]:
# Load posts

# path_posts = "/Users/morales/Desktop/clpsych17-data/data/training/labeled_posts/"
# files = os.listdir(path_posts) 

# post_files = []
# for f in files:
#     pid = f.split('-')[1].replace('.xml','')
#     if pid.strip() in id_labels.keys():
#         post_files.append(f)
        
# posts = {}
# for file in post_files:
#     with open(os.path.join(path_posts,file)) as fd:
#         doc = xmltodict.parse(fd.read())
#         post = doc['response']['message']['body']['#text']
#         string = re.sub("<.*?>", "\n", post)
#         subject = doc['response']['message']['subject']['#text']
#         post_id = doc['response']['message']['id']['#text']
#         posts[post_id] = subject + '\n' + string.replace('\n',' ').strip()

In [ ]:
# Sentence segmentation

# from nltk.tokenize import sent_tokenize

# posts_sents = {}
# for p in posts:
#     post = posts[p]
#     sentences = sent_tokenize(post)
#     posts_sents[p] = sentences
    

In [ ]:
# from nltk.tokenize import TweetTokenizer
# import collections

# tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)

# all_words = []
# for p in posts.keys():
#     label = id_labels[p]
#     if label == 'crisis' or label == 'red':
#         words = tknzr.tokenize(posts[p].lower().replace('.','').replace(',',''))
#         for w in words:
#             all_words.append(w)

# counter = collections.Counter(all_words)
# most = counter.most_common()[:1000]
# top1000 = [pair[0] for pair in most]


In [ ]:
# Get features

# features = []
# labels = []
# lang = "english"
# parser_dir = "/Users/morales/GitHub/models/syntaxnet/"

# for p in posts_sents.keys()[120:]:
#     sentences = posts_sents[p]
#     lab = id_labels[p]
#     labels.append(lab)
#     name = '/Users/morales/Desktop/clpsych17-data/data/training/features3/post_%s_ling.csv' % (p)
#     LA.write_header(name)
#     #Get syntax works at the sentence level
#     print 'This post includes %s sentences' % len(sentences)
#     for s in sentences:
#         LA.get_syntax(s, lang, parser_dir, name)
#     print 'Done processing post %s' % p

In [ ]:
# Combine sentence level features for posts into 1 post 

# feat_files = [f for f in os.listdir(path) if f.endswith('_ling.csv')]
# for f in feat_files:
#     LA.fusion(os.path.join(path,f))

In [43]:
# path = "/Users/morales/Desktop/clpsych17-data/data/training/features2/"
# files = [f for f in os.listdir(path) if f.endswith('_combined.csv')]

path = "/Users/morales/Desktop/clpsych17-data/data/training/features_final/"
files = [f for f in os.listdir(path) if f.endswith('_combined.csv')]
features = []
labels = []
for f in files:
    name = f.split('_')[1]
    l = id_labels[name]
    lines = open(os.path.join(path, f), 'r').readlines()
    labels.append(l)
    features.append([float(x) for x in lines[1].split(',')][242:])

test_ids_wfeats = []
test_path = "/Users/morales/Desktop/clpsych17-data/data/test/"
test_files = [f for f in os.listdir(test_path) if f.endswith('_combined.csv')]
test_features = []
for f in test_files:
    name = f.split('_')[1]
    test_ids_wfeats.append(name)
    lines = open(os.path.join(test_path, f), 'r').readlines()
    feats = [float(x) for x in lines[1].split(',')][242:]
    test_features.append(feats)

# Experiments

In [25]:
# Binary f1 score for urgent (crisis+red) vs. non-urgent (amber+green) posts
labels_bin = []
for l in labels:
    if l=='green' or l=='amber':
        labels_bin.append(0)
    elif l=='crisis' or l=='red':
        labels_bin.append(1)

# Binary f1 score for flagged (crisis+red+amber) vs. non-flagged (green) posts
labels_nogreen = []
for l in labels:
    if l=='green':
        labels_nogreen.append(0)
    else:
        labels_nogreen.append(1)   

# Macro-averaged f1 score, excluding the green (majority) class.
labels_subset = []
features_subset = []
for i, label in enumerate(labels):   
    if label != 'green':
        labels_subset.append(label)
        features_subset.append(features[i])   

In [13]:
# Load test set

path_posts = "/Users/morales/Desktop/clpsych17-data/clpsych17-test/posts/"
files = os.listdir(path_posts) 

# Only consider test posts

with open('/Users/morales/Desktop/clpsych17-data/clpsych17-test/test_ids.tsv') as f:
    test_ids = [l.strip() for l in f.readlines()]
    
post_files = []
for f in files:
    pid = f.split('-')[1].replace('.xml','')
    if pid.strip() in test_ids:
        post_files.append(f)

posts = {}
for file in post_files:
    with open(os.path.join(path_posts,file)) as fd:
        doc = xmltodict.parse(fd.read())
        post = doc['response']['message']['body']['#text']
        string = re.sub("<.*?>", "\n", post)
        subject = doc['response']['message']['subject']['#text']
        post_id = doc['response']['message']['id']['#text']
        posts[post_id] = subject + '\n' + string.replace('\n',' ').strip()


In [ ]:
# Sentence segmentation

from nltk.tokenize import sent_tokenize

path ='/Users/morales/Desktop/clpsych17-data/data/test/'
posts_sents = {}
for p in posts:
    post = posts[p]
    sentences = sent_tokenize(post)
    posts_sents[p] = sentences

# Get features

features = []
lang = "english"
parser_dir = "/Users/morales/GitHub/models/syntaxnet/"

for p in posts_sents.keys()[120:]:
    sentences = posts_sents[p]
    name = '/Users/morales/Desktop/clpsych17-data/data/test/post_%s_ling.csv' % (p)
    LA.write_header(name)
    #Get syntax works at the sentence level
    print 'This post includes %s sentences' % len(sentences)
    for s in sentences:
        LA.get_feats(s, lang, parser_dir, name)
    print 'Done processing post %s' % p

In [22]:
# Combine sentence level features for posts into 1 post 

feat_files = [f for f in os.listdir(path) if f.endswith('_ling.csv')]
for f in feat_files:
    print f
    LA.fusion(os.path.join(path,f))

post_194634_ling.csv
Done combining sentences, new csv saved: /Users/morales/Desktop/clpsych17-data/data/test/post_194634_ling_combined.csv
post_194760_ling.csv
Done combining sentences, new csv saved: /Users/morales/Desktop/clpsych17-data/data/test/post_194760_ling_combined.csv
post_194783_ling.csv
Done combining sentences, new csv saved: /Users/morales/Desktop/clpsych17-data/data/test/post_194783_ling_combined.csv
post_194938_ling.csv
Done combining sentences, new csv saved: /Users/morales/Desktop/clpsych17-data/data/test/post_194938_ling_combined.csv
post_195036_ling.csv
Done combining sentences, new csv saved: /Users/morales/Desktop/clpsych17-data/data/test/post_195036_ling_combined.csv
post_195065_ling.csv
Done combining sentences, new csv saved: /Users/morales/Desktop/clpsych17-data/data/test/post_195065_ling_combined.csv
post_195069_ling.csv
Done combining sentences, new csv saved: /Users/morales/Desktop/clpsych17-data/data/test/post_195069_ling_combined.csv
post_195112_ling.csv

In [44]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, accuracy_score, f1_score

predictions = {}
target_names = ["amber","red","crisis"]


clf = svm.SVC()
clf.fit(features, labels)
y_pred = clf.predict(test_features)
for i, p in enumerate(y_pred):
    predictions[test_ids_wfeats[i]] = p

for tid in test_ids:
    if tid not in predictions.keys():
        predictions[tid] = 'green'

lines = []
for tid in predictions.keys():
    l ='%s\t%s\n' %(tid,predictions[tid])
    lines.append(l)

newf = open('/Users/morales/Desktop/clpsych17-data/submission_sentiment.tsv','w')
for l in lines:
    newf.write(l)
newf.close()

In [26]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, accuracy_score, f1_score

target_names = ['class green+amber', 'class red+crisis']
X_train, X_test, y_train, y_test = train_test_split(features, labels_bin, test_size=0.25, random_state=0)

clf = svm.SVC()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))
print 'Accuracy: ', accuracy_score(y_pred, y_test), '\n'

target_names = ['class green', 'class notgreen']
X_train, X_test, y_train, y_test = train_test_split(features, labels_nogreen, test_size=0.25, random_state=0)

clf = svm.SVC()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))

target_names = ["amber","red","crisis"]
X_train, X_test, y_train, y_test = train_test_split(features_subset, labels_subset, test_size=0.25, random_state=0)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print 'Macro F1-score', f1_score(y_test, y_pred, average='macro') 

                   precision    recall  f1-score   support

class green+amber       0.88      0.99      0.93       260
 class red+crisis       0.25      0.03      0.05        37

      avg / total       0.80      0.87      0.82       297

Accuracy:  0.868686868687 

                precision    recall  f1-score   support

   class green       0.67      0.85      0.75       176
class notgreen       0.64      0.40      0.49       121

   avg / total       0.66      0.66      0.64       297

Macro F1-score 0.254634310752
